# Проект для «Викишоп»

## Введение и постановка задачи

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

**Задача проекта**

Разработать модель, которая сможет классифицировать комментарии на позитивные и негативные. Значение метрики качества F1 должно быть не меньше 0.75.


**Цели проекта**

Для достижения поставленной цели потребуется:
1. Подготовить данные для обучения
2. Выделить признаки
3. Провести лемматизацию
4. Провести векторизацию
5. Обучить несколько моделей
6. Сравнить обученные модели и выбрать наилучшую
7. Сделать выводы.

## Подготовка

Импортируем необходимые библиотеки

In [233]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_curve
from tqdm import notebook
from tqdm.notebook import tqdm
tqdm.pandas()
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from catboost import CatBoostClassifier
from sklearn.utils.class_weight import compute_class_weight
import joblib

Прочитаем файл, предварительно скопировав его в каталог с тетрадкой.

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

Выведем информацию о таблице

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


В таблице 159571 записей без пропусков. В столбце `text` сам комментарий, в столбце `toxic` - оценка коментария - целевой признак. Выведем первые строки.

In [4]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Переведём текст в нижний регистр

In [3]:
df['text'] = df['text'].str.lower()

Все не буквенные символы заменим пробелами

In [4]:
df['text'] = df['text'].apply(lambda x: ' '.join(re.sub(r'[^a-zA-Z ]', ' ', x).split()))

Проверим на дубликаты

In [5]:
df.duplicated().sum()

1293

In [6]:
df['text'].duplicated().sum()

1321

Любопытно. Разность дубликатов во всём фрейме и в столбце `text` может говорить о том, что одинаковые комментарии признаются как токсичными, так и не токсичными. Таких комментариев 28. Возможно в этом есть какой-то смысл. Удалим те комментарии, которые полностью дублируются.

In [7]:
df = df.drop_duplicates().reset_index(drop = True)

Проверим баланс классов

In [8]:
df['toxic'].value_counts(normalize=True)

0    0.898261
1    0.101739
Name: toxic, dtype: float64

Видно, что токсичных комментариев только 10%. Надо это учесть при обучении модели.

### Выводы
Файл открылся без проблем. Столбцы соответствуют заявленным. Пропусков нет. Дубликаты удалили. Наблюдается дисбаланс классов.

## Обучение

### Выделение признаков

Сначала обозначим признаки и таргет

In [9]:
X = df.copy()

In [10]:
y = X.pop('toxic')

### Лемматизация

In [14]:
m = Mystem()

Проведём лемматизацию текста с использованием прогресс-бара

In [15]:
lemmatized = X['text'].progress_apply(lambda x: m.lemmatize(x))

  0%|          | 0/158278 [00:00<?, ?it/s]

Из списка переведём текст в строку 

In [16]:
joined = lemmatized.progress_apply(lambda x: ''.join(x))

  0%|          | 0/158278 [00:00<?, ?it/s]

Загрузим стоп-слова для английского языка

In [ ]:
nltk.download('stopwords')

In [234]:
stopwords = set(nltk_stopwords.words('english'))

### Векторизация текста

Для векторизации используем значение TF-IDF. Воспользуемся `TfidfVectorizer` библиотеки `sklearn`.

In [18]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 

Разделим объединённые признаки в `joined` и `y` на тренировочную и тестовую выборки

In [18]:
X_train, X_test, y_train, y_test = train_test_split(joined, y, test_size = 0.2, random_state = 42)

In [19]:
len(X_train), len(X_test), len(y_train), len(y_test)

(126622, 31656, 126622, 31656)

Проведём обучение на тренировочном наборе

In [21]:
count_tf_idf.fit(X_train)

TfidfVectorizer(stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})

Трансформируем пренировочные признаки

In [33]:
X_train_t = count_tf_idf.transform(X_train)

Посмотрим на размеры

In [34]:
X_train_t.shape

(126622, 148069)

Признаки подготовлены. Далее проведём обучение моделей:
- Логистическая регрессия
- Случайный лес
- CatBostClassifier

In [11]:
models_name = ['LogisticRegression', 'RandomForest', 'CatBoost']

Сами модели запишем отдельно

In [60]:
models = []

Результаты обучения также будем записывать, для итоговой статистики

In [61]:
f1_score_train = []

In [77]:
f1_score_test = []

### Логистическая регресиия

Создадим модель, учтём, что классы необходимо отбалансировать

In [23]:
log_model = LogisticRegression(class_weight = 'balanced')

Зададим несколько параметров для перебора

In [33]:
log_param = {'fit_intercept':[True, False]}

Параметры будем перебирать по сетке

In [34]:
log_search = GridSearchCV(log_model, param_grid = log_param, scoring = 'f1', cv=5, verbose=1)

Запустим

In [35]:
log_search.fit(X_train_t, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

GridSearchCV(cv=5, estimator=LogisticRegression(class_weight='balanced'),
             param_grid={'fit_intercept': [True, False]}, scoring='f1',
             verbose=1)

Выведем результаты

In [37]:
pd.DataFrame(log_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_fit_intercept,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,61.995493,9.115324,0.132740,0.036615,True,{'fit_intercept': True},0.754167,0.746107,0.749472,0.755002,0.753797,0.751709,0.003395,1
1,21.098064,0.869021,0.100295,0.055210,False,{'fit_intercept': False},0.706883,0.706927,0.706558,0.716136,0.712145,0.709730,0.003818,2


Видим, что лучший результат показала модель с `'fit_intercept'= True`, хоть время обучения и больше. Метрика чуть больше 0.75, что в принципе нас устраивает. Будем использовать эту модель

In [38]:
log_model = log_search.best_estimator_

Запишем модель

In [64]:
models.append(log_model)

Получим значение метрики на всём тренировочном датасете

In [65]:
f1_score(y_train, log_model.predict(X_train_t))

0.8405963302752294

In [66]:
f1_score_train.append(f1_score(y_train, log_model.predict(X_train_t)))

### Случайный лес

Следующей моделью используем случайны лес. Ограничим к-во деревьев для ускорения расчёта

In [35]:
forest_model = RandomForestClassifier(random_state=42, class_weight = 'balanced')

Зададим параметры 

In [36]:
forest_params = {'n_estimators':[10,20]} 

Поиск по гриду

In [37]:
forest_search = GridSearchCV(forest_model, param_grid = forest_params, scoring = 'f1', cv=5, verbose=1)

In [38]:
forest_search.fit(X_train_t, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=42),
             param_grid={'n_estimators': [10, 20]}, scoring='f1', verbose=1)

Выведем результат

In [39]:
pd.DataFrame(forest_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,84.250841,0.783575,0.291079,0.002867,10,{'n_estimators': 10},0.600157,0.594580,0.593985,0.594499,0.577755,0.592195,0.007564,2
1,173.110507,1.672615,0.575899,0.013353,20,{'n_estimators': 20},0.616701,0.615946,0.612204,0.622049,0.613237,0.616027,0.003439,1


Результат хучше, чем для регресии. Вероятно слишком мало деревьев.

In [40]:
forest_model = forest_search.best_estimator_

Запишем модель

In [67]:
models.append(forest_model)

Получим значение метрики на всём тренировочном датасет

In [68]:
f1_score_train.append(f1_score(y_train, forest_model.predict(X_train_t)))

In [43]:
f1_score(y_train, forest_model.predict(X_train_t))

0.9880812357876577

### CatBoostClassifier

Несмотря на то, что у модели есть возможность работать с текстом напрямую, проведём поиск на подготовленных признаках, и удем использовать параметры поиска аналогичный лесу. Предварительно определим веса классов для балансировки

In [47]:
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

In [48]:
class_weights

{0: 0.5565900059781271, 1: 4.917741183781264}

Создадим модель

In [49]:
cat_model = CatBoostClassifier(random_state = 42,class_weights = class_weights)

Перебор по сетке

In [50]:
cat_search = GridSearchCV(cat_model, param_grid = forest_params, scoring = 'f1', cv=5, verbose=1)

Запустим поиск

In [51]:
cat_search.fit(X_train_t, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Learning rate set to 0.5
0:	learn: 0.5634541	total: 3s	remaining: 27s
1:	learn: 0.5268993	total: 5.02s	remaining: 20.1s
2:	learn: 0.4931246	total: 6.98s	remaining: 16.3s
3:	learn: 0.4742579	total: 8.94s	remaining: 13.4s
4:	learn: 0.4587846	total: 10.9s	remaining: 10.9s
5:	learn: 0.4421423	total: 12.8s	remaining: 8.53s
6:	learn: 0.4311860	total: 14.7s	remaining: 6.31s
7:	learn: 0.4223865	total: 16.7s	remaining: 4.16s
8:	learn: 0.4135801	total: 18.5s	remaining: 2.06s
9:	learn: 0.4044771	total: 20.4s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.5663680	total: 2.88s	remaining: 25.9s
1:	learn: 0.5188445	total: 4.87s	remaining: 19.5s
2:	learn: 0.4930166	total: 6.84s	remaining: 16s
3:	learn: 0.4691810	total: 8.74s	remaining: 13.1s
4:	learn: 0.4545953	total: 10.6s	remaining: 10.6s
5:	learn: 0.4419269	total: 12.5s	remaining: 8.35s
6:	learn: 0.4305902	total: 14.5s	remaining: 6.22s
7:	learn: 0.4226455	total: 16.5s	remaining: 4.12s

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x7f8cc8b8e580>,
             param_grid={'n_estimators': [10, 20]}, scoring='f1', verbose=1)

Выведем результат

In [52]:
pd.DataFrame(cat_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,48.792274,0.670024,0.267991,0.003549,10,{'n_estimators': 10},0.656700,0.650354,0.642646,0.668854,0.673205,0.658352,0.011352,2
1,67.549397,0.515699,0.275307,0.006982,20,{'n_estimators': 20},0.701819,0.692756,0.699462,0.704087,0.691954,0.698016,0.004854,1


Результат хучше, чем для регресии, но лучше чем для леса. Вероятно слишком мало деревьев.

In [53]:
cat_model = cat_search.best_estimator_

Запишем модель

In [69]:
models.append(cat_model)

Получим значение метрики на всём тренировочном датасет

In [70]:
f1_score_train.append(f1_score(y_train, cat_model.predict(X_train_t)))

In [56]:
f1_score(y_train, cat_model.predict(X_train_t))

0.7023306627822287

### Выводы

На тренировочном датасете лучший результат показал случайный лес. Затем регрессия и CatBoost. CatBoost не показал требуемую метрику, а метрика случайного леса слишком хороша, возможно лес переобучился. Проверим это на тесте.

## Тестирование модели

Проведём тестирование моделей на тестировочной выборке и построим ROC AUC. Предварительно преобразуем признаки

In [73]:
X_test_t = count_tf_idf.transform(X_test)

In [74]:
X_test_t.shape

(31656, 148069)

Запустим цикл с моделями и рассчитаем для каждой модели из списка метрику F1 для тестовой выборки

In [79]:
for model in models:
    f1_score_test.append(f1_score(y_test, model.predict(X_test_t)))

Представим результат в удобном виде

In [149]:
pd.DataFrame({'models_name':models_name, 'f1_score_train':f1_score_train, 'f1_score_test':f1_score_test}).sort_values(
    by='f1_score_test', ascending=False)

,models_name,f1_score_train,f1_score_test
0,LogisticRegression,0.840596,0.756951
2,CatBoost,0.702331,0.686848
1,RandomForest,0.988081,0.618340


### Выводы

Из таблицы видно, что лучший результат на тестовой выборке показала логистическая регрессия, затем `CatBoost` и на последнем месте СлучайныйЛес. Все модели уходшили свои показатели, однако наибольшую разницу по показал Случайный лес.

## Общие выводы

В ходе работы над проектом мы подготовили данные для обучения - удалили дубликаты и небуквенные символы. Перевели в нижний регистр. Выделели признаки и провели лемматизацию. Векторизацию провели используя *TfidfVectorizer*. Для обучения выбрали Логистическую регрессию, Случайный лес и CatBostClassifier. Грид-серчем нашли лучшие параметры для них. Поиск проводили по одному параметру, чтобы не увеличивать время. На тренировочной выборке лучшую метрику показал Случайный лес, однако на тесте лучшей моделью стала Логистическая регрессия. Эту модель и будем рекомендовать для использования в классификации комментариев.